In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import RidgeCV
import random
import pickle as pkl

In [22]:
random.seed(24)

In [23]:
eco_fac = pd.read_csv('eco_canada.csv')
eco_fac.head()

,,GDP_US,INFL_US,GDP,PPP,INFL,FER
0,1981,13976.1,10.334710,12337.5,1.134384,12.471610,1.198903
1,1982,14433.8,6.131427,12481.9,1.162945,10.768970,1.233735
2,1983,15543.9,3.212435,13425.1,1.183843,5.863588,1.232412
3,1984,17121.2,4.300536,13877.9,1.183011,4.304778,1.295066
4,1985,18236.8,3.545644,14114.8,1.185028,3.962031,1.365507


In [24]:
df = pd.read_csv('weights_15.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.025855,0.125928,0.054118,0.067509,0.037793,0.062103,0.057768,0.093079,0.032104,0.069524,0.053799,0.089232,0.020576,0.195674,0.014940
1,0.024390,0.107464,0.049101,0.059169,0.034891,0.060548,0.056063,0.094063,0.027917,0.074106,0.045950,0.088189,0.022177,0.239667,0.016304
2,0.025450,0.076065,0.035598,0.047638,0.028261,0.041214,0.046687,0.079440,0.030794,0.057839,0.034091,0.071603,0.016388,0.389751,0.019181
3,0.026685,0.087435,0.042064,0.061242,0.031366,0.051220,0.059974,0.104375,0.038531,0.068460,0.035101,0.074283,0.016077,0.283385,0.019803
4,0.028514,0.077057,0.046638,0.060663,0.031233,0.060692,0.060507,0.107761,0.041098,0.076824,0.038589,0.085528,0.015696,0.251406,0.017794


In [25]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

In [26]:
new_fer = eco_fac.FER.shift(-1)
new_fer = pd.DataFrame(new_fer)
new_fer.rename(columns = {'FER':'NEXT_YR_FER'}, inplace=True)

In [27]:
data = pd.concat([eco_fac,new_fer],axis=1)
data.head()

,,GDP_US,INFL_US,GDP,PPP,INFL,FER,NEXT_YR_FER
0,1981,13976.1,10.334710,12337.5,1.134384,12.471610,1.198903,1.233735
1,1982,14433.8,6.131427,12481.9,1.162945,10.768970,1.233735,1.232412
2,1983,15543.9,3.212435,13425.1,1.183843,5.863588,1.232412,1.295066
3,1984,17121.2,4.300536,13877.9,1.183011,4.304778,1.295066,1.365507
4,1985,18236.8,3.545644,14114.8,1.185028,3.962031,1.365507,1.389471


In [29]:
data = pd.concat([df,data],axis=1)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,GDP,FER,NEXT_YR_FER
0,0.025855,0.125928,0.054118,0.067509,0.037793,0.062103,0.057768,0.093079,0.032104,0.069524,0.053799,0.089232,0.020576,0.195674,0.014940,12337.5,1.198903,1.233735
1,0.024390,0.107464,0.049101,0.059169,0.034891,0.060548,0.056063,0.094063,0.027917,0.074106,0.045950,0.088189,0.022177,0.239667,0.016304,12481.9,1.233735,1.232412
2,0.025450,0.076065,0.035598,0.047638,0.028261,0.041214,0.046687,0.079440,0.030794,0.057839,0.034091,0.071603,0.016388,0.389751,0.019181,13425.1,1.232412,1.295066
3,0.026685,0.087435,0.042064,0.061242,0.031366,0.051220,0.059974,0.104375,0.038531,0.068460,0.035101,0.074283,0.016077,0.283385,0.019803,13877.9,1.295066,1.365507
4,0.028514,0.077057,0.046638,0.060663,0.031233,0.060692,0.060507,0.107761,0.041098,0.076824,0.038589,0.085528,0.015696,0.251406,0.017794,14114.8,1.365507,1.389471


In [30]:
data = data.dropna()

In [31]:
data.shape

(38, 18)

In [32]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [34]:
minmax = MinMaxScaler()
scaler = StandardScaler()

In [35]:
X_s = scaler.fit_transform(X) # standard scaled data
X_m = minmax.fit_transform(X) # minmax scaled data

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)
X_train_s, X_test_s, y_train, y_test = train_test_split(X_s, y, test_size=0.2, random_state=None)
X_train_m, X_test_m, y_train, y_test = train_test_split(X_m, y, test_size=0.2, random_state=None)

In [37]:
# Change model here

from sklearn.model_selection import cross_val_score
score = cross_val_score(RidgeCV(), X_train, y_train, cv=5, scoring='neg_mean_squared_error')
score.mean()

-0.02861038476596947

In [38]:
# Change model here

score = cross_val_score(RidgeCV(), X_train_m, y_train, cv=5, scoring='neg_mean_squared_error')
score.mean()

-0.0032023941864332433

In [39]:
# Change model here

score = cross_val_score(RidgeCV(), X_train_s, y_train, cv=5, scoring='neg_mean_squared_error')
score.mean()

-0.08476447290118957

In [40]:
# Original 
model_orig = RidgeCV() # change model here
model_orig.fit(X_train, y_train)
pred = model_orig.predict(X_test)
print('MSE :',mean_squared_error(pred, y_test))
print('R2 :',r2_score(pred,y_test))
print('Error : ',((y_test-pred)/y_test).mean())

MSE : 0.009073043133819257
R2 : -13.33467204974958
Error :  0.04324051760980878


In [41]:
(y_test-pred) # check the variations

22    0.029070
37    0.056901
15    0.111209
7    -0.102284
34    0.076900
13    0.085401
33    0.055073
4     0.171558
Name: NEXT_YR_FER, dtype: float64

In [42]:
# MinMax Scaled 
model_minmax = RidgeCV() # Change model here
model_minmax.fit(X_train_m, y_train)
pred = model_minmax.predict(X_test_m)
print('MSE :',mean_squared_error(pred, y_test))
print('R2 :',r2_score(pred,y_test))
print('Error : ',((y_test-pred)/y_test).mean())

MSE : 0.009158598325970659
R2 : 0.14754563614629324
Error :  0.03456468916959386


In [43]:
(y_test-pred) # check the variation

22    0.093811
37    0.008373
15   -0.002179
7    -0.064127
34    0.179892
13   -0.041492
33    0.158186
4     0.034292
Name: NEXT_YR_FER, dtype: float64

In [44]:
# Standard Scaled 
model_stand = RidgeCV() # Change model here
model_stand.fit(X_train_s, y_train)
pred = model_stand.predict(X_test_s)
print('MSE :',mean_squared_error(pred, y_test))
print('R2 :',r2_score(pred,y_test))
print('Error : ',((y_test-pred)/y_test).mean())

MSE : 0.012882518710479744
R2 : -3.104638533127331
Error :  0.04599085962116305


In [45]:
(y_test-pred) # check the variations

22    0.003420
37   -0.036653
15    0.195412
7    -0.064967
34    0.022090
13    0.158690
33    0.065264
4     0.171373
Name: NEXT_YR_FER, dtype: float64

In [48]:
# Save the best perform ming model only after instructions

pkl.dump(model_minmax, open('model_canada.pkl','wb'))

In [49]:
# dont uncomment until said to
# if we use scaling
pkl.dump(minmax, open('scaler_canada.pkl','wb'))